In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import input_file_name, lit

In [12]:
#Create a SparkSession
spark = SparkSession.builder.appName("ETFs").getOrCreate()

In [18]:


existing_schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", FloatType(), True),
    StructField("High", FloatType(), True),
    StructField("Low", FloatType(), True),
    StructField("Close", FloatType(), True),
    StructField("Adj Close", FloatType(), True),
    StructField("Volume", FloatType(), True)

])

In [42]:
input_path = "../data/stocks_etfs/*.csv"
stock_df = spark.read.csv(input_path, header=True, schema=existing_schema)

In [43]:
stock_df.count()

97840

In [44]:
stock_df.tail(20)

[Row(Date='2020-03-05', Open=1.0, High=1.0, Low=0.9599999785423279, Close=0.9599999785423279, Adj Close=0.9599999785423279, Volume=2600.0),
 Row(Date='2020-03-06', Open=0.9599999785423279, High=1.0099999904632568, Low=0.9599999785423279, Close=1.0, Adj Close=1.0, Volume=8900.0),
 Row(Date='2020-03-09', Open=1.0, High=1.0299999713897705, Low=0.9700000286102295, Close=0.9700000286102295, Adj Close=0.9700000286102295, Volume=23700.0),
 Row(Date='2020-03-10', Open=0.9800000190734863, High=1.0199999809265137, Low=0.9599999785423279, Close=0.9800000190734863, Adj Close=0.9800000190734863, Volume=9100.0),
 Row(Date='2020-03-11', Open=0.9599999785423279, High=0.9700000286102295, Low=0.9599999785423279, Close=0.9599999785423279, Adj Close=0.9599999785423279, Volume=35600.0),
 Row(Date='2020-03-12', Open=0.9599999785423279, High=1.0, Low=0.8999999761581421, Close=0.8999999761581421, Adj Close=0.8999999761581421, Volume=35900.0),
 Row(Date='2020-03-13', Open=0.8999999761581421, High=0.97000002861

In [47]:
stock_df.show()

+----------+--------+--------+--------+--------+---------+--------+
|      Date|    Open|    High|     Low|   Close|Adj Close|  Volume|
+----------+--------+--------+--------+--------+---------+--------+
|1962-01-02|6.532155|6.556185|6.532155|6.532155|1.5366576| 55900.0|
|1962-01-03|6.532155| 6.63228|6.524145| 6.63228|1.5602118| 74500.0|
|1962-01-04| 6.63228| 6.66432| 6.63228| 6.63228|1.5602118| 80500.0|
|1962-01-05| 6.63228| 6.65631| 6.61626| 6.62427|1.5583265| 70500.0|
|1962-01-08| 6.60825| 6.60825|6.339915|   6.408|1.5074503| 93800.0|
|1962-01-09|   6.408|6.508125|6.355935|6.355935|1.4952024| 64500.0|
|1962-01-10|6.355935|   6.408|6.339915|6.363945|1.4970871| 34500.0|
|1962-01-11|6.347925|6.347925|6.323895|6.339915|1.4914336| 27900.0|
|1962-01-12|6.339915|6.347925| 6.20775| 6.20775|1.4603431| 26500.0|
|1962-01-15|6.139665|6.139665| 6.05556| 6.05556|1.4245404| 64500.0|
|1962-01-16| 6.03954| 6.03954|  6.0075|  6.0075|1.4132345| 27900.0|
|1962-01-17|  6.0075| 6.02352|  6.0075|  6.0075|

In [8]:
import pandas as pd

pd.read_csv("../data/stocks_etfs/A.csv")

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200
...,...,...,...,...,...,...,...
5119,2020-03-26,70.000000,74.449997,69.650002,73.720001,73.532867,3267500
5120,2020-03-27,71.550003,73.209999,70.279999,70.910004,70.730003,1829800
5121,2020-03-30,71.059998,73.180000,71.059998,72.669998,72.669998,1486200
5122,2020-03-31,72.339996,72.800003,70.500000,71.620003,71.620003,1822100
